In [2]:
from mushroom import *

c:\Users\Melissa Wang\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...: 100%|██████████| 9/9 [00:11<00:00,  1.30s/it]
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support fo

In [ ]:
root_folder = "mushrooms"
batch_size = 8

embeds_path = "embeds_all.npy"
gt_labels_path = "ground_truth_labels.npy"
pred_labels_path = "predicted_labels.npy"
species_names_path = "species_names.npy"
image_paths_path = "image_paths.npy"

if all(os.path.exists(p) for p in [embeds_path, gt_labels_path, pred_labels_path, species_names_path, image_paths_path]):
    all_embeds = np.load(embeds_path)
    ground_truth_labels = np.load(gt_labels_path)
    pred_labels = np.load(pred_labels_path)
    species_names = np.load(species_names_path, allow_pickle=True)
    image_paths = np.load(image_paths_path, allow_pickle=True)
    print("All data loaded from saved files.")
else:
    all_embeds = []
    ground_truth_labels = []
    image_paths = []

    for species in os.listdir(root_folder):
            species_folder = os.path.join(root_folder, species)
            if os.path.isdir(species_folder):
                files = sorted(f for f in os.listdir(species_folder) if f.lower().endswith(".png"))
                paths = [os.path.join(species_folder, f) for f in files]
                image_paths.extend(paths)
                ground_truth_labels.extend([species] * len(files))
                embeds = embed_whole_folder(species_folder, bs=batch_size)  # your embedding function
                all_embeds.append(embeds)

    all_embeds = np.vstack(all_embeds)
    all_embeds = normalize_embeds(all_embeds)
    ground_truth_labels = np.array(ground_truth_labels)
    species_names = list(np.unique(ground_truth_labels))
    pred_labels = classify_images(pipe, image_paths, species_names)

    np.save(embeds_path, all_embeds)
    np.save(gt_labels_path, ground_truth_labels)
    np.save(pred_labels_path, pred_labels)
    np.save(species_names_path, species_names)
    np.save(image_paths_path, image_paths)

    print("All embeddings, labels, and predictions computed and saved.")





In [ ]:
#embedding all images once and saving 
root_folder = "mushrooms"        # main folder containing species subfolders
batch_size = 8                   # batch size for embedding
save_path = "embeds_all.npy"

all_embeds = []

for species in os.listdir(root_folder):
    species_folder = os.path.join(root_folder, species)
    if os.path.isdir(species_folder):
        embeds = embed_whole_folder(species_folder, bs=batch_size)
        all_embeds.append(embeds)

all_embeds = np.vstack(all_embeds)
all_embeds = normalize_embeds(all_embeds)
np.save(save_path, all_embeds)


In [ ]:
root_folder = "mushrooms"        # main folder containing species subfolders
batch_size = 8         
ground_truth_labels = []
image_paths = []
for species in os.listdir(root_folder):
    species_folder = os.path.join(root_folder, species)
    if os.path.isdir(species_folder):
        files = [f for f in sorted(os.listdir(species_folder)) if f.lower().endswith(".png")]
        img_count = len(files)
        image_paths.extend([os.path.join(species_folder, f) for f in files])    
        ground_truth_labels.extend([species] * img_count)

ground_truth_labels = np.array(ground_truth_labels)

species_names = list(np.unique(ground_truth_labels))
pred_labels = classify_images(pipe, image_paths, species_names)

np.save("predicted_labels.npy", pred_labels)
np.save("ground_truth_labels.npy", ground_truth_labels)



First 10 predicted labels: [np.str_('fly_agaric'), np.str_('fly_agaric'), np.str_('fly_agaric'), np.str_('beefsteak_fungus'), np.str_('fly_agaric'), np.str_('fly_agaric'), np.str_('fly_agaric'), np.str_('fly_agaric'), np.str_('fly_agaric'), np.str_('fly_agaric')]


In [1]:
num_clusters = len(species_names)

kmeans_labels, kmeans_model = cluster(all_embeds, n=num_clusters)
gmm_labels, gmm_probs, gmm_model = cluster_gmm(all_embeds, k=num_clusters)
agg_labels, agg_model = cluster_agglomerative(all_embeds, k=num_clusters)
spec_labels, spec_model = cluster_spectral(all_embeds, k=num_clusters)
dbscan_labels, dbscan_model = cluster_dbscan(all_embeds, eps=0.5, min_samples=3)

NameError: name 'species_names' is not defined

In [12]:
print("KMeans silhouette score:", cluster_quality(all_embeds, kmeans_labels))
print("GMM silhouette score:", cluster_quality(all_embeds, gmm_labels))
print("Agglomerative silhouette score:", cluster_quality(all_embeds, agg_labels))
print("DBSCAN silhouette score:", cluster_quality(all_embeds, dbscan_labels))
print("Spectral silhouette score:", cluster_quality(all_embeds, spec_labels))


KMeans silhouette score: 0.08680424300224751
GMM silhouette score: 0.08680424300224751
Agglomerative silhouette score: 0.0888465861950367
DBSCAN silhouette score: 0.09936613767276346
Spectral silhouette score: 0.0949823892819439


Hmmm these scores are all very low.. Clusters are not well-separated in the embedding space. DBSCAN highest score... so desnity-based clustering better than kmeans? Kmeans and GMM identical so embeddings are all gaussian. agglomerative and spectral only slightly better. but Spectral does have the second highest so maybe there is some more organic cluster shapes eg swirls/moons. 